# Atividade 5

Implementação do K-fold Cross Validation para estimar os erros de estimação dos classificadores que iremos utilizar nas atividades futuras.

## 0. Preparação do ambiente.
Para os trabalhos desta atividade utilizaremos a base de dados previamente carregada no Github, [neste link](https://github.com/danielbdias/pattern-recognition-studies.git), em conjunto com alguns scripts auxiliares

In [ ]:
# prepara a máquina local do google colab para receber a base (quando necessário)
# e baixa os scripts auxiliares para a montagem do notebook
!rm -rf ./*

!git clone https://github.com/danielbdias/pattern-recognition-studies.git
!mv ./pattern-recognition-studies/* ./
!rm -rf ./pattern-recognition-studies
!rm -rf ./sample_data

Cloning into 'pattern-recognition-studies'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 73 (delta 24), reused 70 (delta 21), pack-reused 0
Unpacking objects: 100% (73/73), done.


In [1]:
# imports de libs necessárias para as análises
from scripts.database import load_datapoints_with_targets
from scripts.preprocessing import centralize_observations, transform_to_distances

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold

## 1. Base de dados

Para o nosso trabalhos, estamos utilizando o dataset [Grammatical Facial Expressions](https://archive.ics.uci.edu/ml/datasets/Grammatical+Facial+Expressions), que descreve expressões faciais gramaticais da linguagem brasileira de sinais (Libras).

A base possui `27965 instâncias`, subdivididas em 9 expressões: `Interrogativa (qu)`,`Interrogativa (s/n)`, `Interrogativa (dúvida)`, `Negativa`, `Afirmativa`, `Condicional`, `Relativa`, `Tópico` e `Foco`. 

Cada instância é estruturada em `300 características`, que representam 100 pontos com coordenadas (x, y, z) da face. Não há missing values nesses pontos e eles não por um processo de normalização.

A classificação de instância é binária onde ela pode ser "Com Expressão" ("Expression", onde os pontos representam a expressão facial) ou "Sem Expressão" ("Not Expression", os pontos não representam uma expressão).

Para esta atividade e as próximas, optamos por utilizar utilizar somente as observações da expressão facial `negativa`, primeiro em sua forma bruta e depois pré-processada.

### 1.1 Base "bruta"

A base com os dados brutos (sem tratamentos) apresenta as seguintes estatísticas descritivas:

In [2]:
category = 'negative'
raw_data = load_datapoints_with_targets(category)

raw_data.describe()

,0x,0y,0z,1x,1y,1z,2x,2y,2z,3x,...,97x,97y,97z,98x,98y,98z,99x,99y,99z,target
count,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,...,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000
mean,309.397827,219.230181,1060.284922,306.747875,217.450552,1131.164819,303.378426,216.894622,1193.764597,300.074385,...,336.213900,205.329225,1272.370658,341.628362,207.474289,1184.382114,344.936643,210.218903,986.881744,0.458241
std,10.612048,14.874460,439.370887,10.614432,15.017886,361.558210,10.643030,15.185494,266.930040,10.749632,...,8.731956,13.120294,89.417381,8.828361,12.035667,351.531081,9.283779,11.350090,555.178027,0.498345
min,284.689000,180.156000,0.000000,281.267000,177.075000,0.000000,276.624000,176.110000,0.000000,272.141000,...,316.827000,160.573000,0.000000,321.877000,164.759000,0.000000,325.012000,169.715000,0.000000,0.000000
25%,298.539250,201.938750,1216.000000,295.935750,199.979500,1221.000000,292.494250,199.242250,1225.000000,288.968250,...,328.490000,190.455000,1265.000000,333.771500,193.954250,1275.000000,336.410250,197.584000,1208.000000,0.000000
50%,312.030000,229.251000,1238.000000,309.420500,227.463000,1243.000000,306.307000,227.081500,1256.000000,303.322500,...,335.835000,213.641500,1284.000000,341.306000,214.365500,1294.000000,345.151000,216.028000,1294.000000,0.000000
75%,317.974750,231.949000,1261.000000,315.219000,230.355250,1270.000000,311.805000,229.977750,1280.000000,308.526750,...,342.709500,216.596250,1294.000000,348.130500,217.373750,1299.000000,351.981750,219.523500,1309.000000,1.000000
max,331.248000,239.845000,1299.000000,328.514000,238.110000,1304.000000,325.127000,237.517000,1314.000000,321.947000,...,356.602000,223.180000,1329.000000,362.141000,223.940000,1542.000000,365.228000,225.527000,1563.000000,1.000000


### 1.2 Base normalizada

No processo de normalização da base buscamos centralizar cada frame (observação) em relação a um ponto em comum no frame.
Escolhemos o ponto 89 (`nose tip`) como referência e aplicamos o processo nas seguintes etapas:

1.   Encontramos os valores médios para o ponto 89 nos valores de `x` e `y` (ignoramos o valor de `z` por ele usar uma medida em milimetros ao invés de em pixels, [referência](https://archive.ics.uci.edu/ml/datasets/Grammatical+Facial+Expressions#));
2.   Calculamos os deltas do ponto 89 de cada observação em relação aos valores médios encontrados;
3.   Subtraimos esse delta de todos os pontos de cada observação.

Aplicando essa normalização percebemos que a média permaneceu a mesma, mas a amplitude dos dados diminuiu nas bases, o que pode ser constatado pela diferença entre o mínimo e o máximo de cada variável.

In [3]:
normalized_data = centralize_observations(raw_data)
normalized_data.describe()

,0x,0y,0z,1x,1y,1z,2x,2y,2z,3x,...,97x,97y,97z,98x,98y,98z,99x,99y,99z,target
count,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,...,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000,2706.000000
mean,309.397827,219.230181,1060.284922,306.747875,217.450552,1131.164819,303.378426,216.894622,1193.764597,300.074385,...,336.213900,205.329225,1272.370658,341.628362,207.474289,1184.382114,344.936643,210.218903,986.881744,0.458241
std,1.625597,2.741896,439.370887,1.766325,2.647455,361.558210,1.909747,2.545192,266.930040,2.068360,...,3.905090,5.389459,89.417381,3.914957,6.521451,351.531081,3.385256,7.330508,555.178027,0.498345
min,303.950854,209.797120,0.000000,300.704854,207.540120,0.000000,296.182854,206.904120,0.000000,291.035854,...,324.399854,191.588120,0.000000,329.433854,190.305120,0.000000,333.625854,190.429120,0.000000,0.000000
25%,308.392854,217.153620,1216.000000,305.627854,215.371120,1221.000000,302.096854,214.948620,1225.000000,298.725104,...,332.981354,200.486870,1265.000000,338.390354,201.331120,1275.000000,342.261354,203.145120,1208.000000,0.000000
50%,309.606354,219.179620,1238.000000,306.972854,217.450620,1243.000000,303.588354,217.093620,1256.000000,300.290354,...,335.892854,206.155620,1284.000000,341.439354,208.103120,1294.000000,345.075854,210.954620,1294.000000,0.000000
75%,310.441854,221.674620,1261.000000,307.857104,219.733870,1270.000000,304.494854,219.018620,1280.000000,301.115854,...,340.044854,210.090870,1294.000000,345.416854,213.098120,1299.000000,348.082604,216.895370,1309.000000,1.000000
max,314.504854,224.502120,1299.000000,312.710854,222.920120,1304.000000,310.230854,222.413120,1314.000000,307.853854,...,348.181854,214.912120,1329.000000,356.601854,219.077120,1542.000000,360.888854,223.073120,1563.000000,1.000000


## 2. Validação Cruzada

Função que imprime info sobre os folds

In [4]:
def compute_statistics(classes):
  n_classes = len(classes)
  n_positive = len(classes[classes == 1])
  n_negative = len(classes[classes == 0])
  prop_positive = round(n_positive / n_classes, 3)
  prop_negative = round(1.0 - prop_positive, 3)

  return n_classes, n_positive, n_negative, prop_positive, prop_negative

def Stratified_KFold(X, y, K):
  kf = StratifiedKFold(n_splits=K, shuffle=False)
  
  n_classes, n_pos, n_neg, prop_pos, prop_neg = compute_statistics(y)    
  print(f'proporção inicial: pos {prop_pos} neg {prop_neg}')

  train_and_test_indexes = kf.split(X, y)

  for i, indexes in enumerate(train_and_test_indexes, 1):
    train_index, test_index = indexes

    X1_train, X1_test = X[train_index], X[test_index]
    y1_train, y1_test = y[train_index], y[test_index]
    
    n_treino, n_pos_treino, n_neg_treino, prop_pos_treino, prop_neg_treino = compute_statistics(y1_train)    
    n_teste, n_pos_teste, n_neg_teste, prop_pos_teste, prop_neg_teste = compute_statistics(y1_test)
    
    print(f'Fold: {i} nº treino {n_treino} nº pos treino {n_pos_treino} nº neg treino {n_neg_treino} proporção treino pos {prop_pos_treino} neg {prop_neg_treino} Total: {n_classes}')
    print(f'Fold: {i} nº teste {n_teste} nº pos teste {n_pos_teste} nº neg teste {n_neg_teste} proporção teste pos {prop_pos_teste} neg {prop_neg_teste} Total: {n_classes}')
    
  return K

In [5]:
X = normalized_data.values[:,:-1]
y = normalized_data.values[:, -1:].ravel()

Stratified_KFold(X, y, K = 10)

proporção inicial: pos 0.458 neg 0.542
Fold: 1 nº treino 2435 nº pos treino 1116 nº neg treino 1319 proporção treino pos 0.458 neg 0.542 Total: 2706
Fold: 1 nº teste 271 nº pos teste 124 nº neg teste 147 proporção teste pos 0.458 neg 0.542 Total: 2706
Fold: 2 nº treino 2435 nº pos treino 1116 nº neg treino 1319 proporção treino pos 0.458 neg 0.542 Total: 2706
Fold: 2 nº teste 271 nº pos teste 124 nº neg teste 147 proporção teste pos 0.458 neg 0.542 Total: 2706
Fold: 3 nº treino 2435 nº pos treino 1116 nº neg treino 1319 proporção treino pos 0.458 neg 0.542 Total: 2706
Fold: 3 nº teste 271 nº pos teste 124 nº neg teste 147 proporção teste pos 0.458 neg 0.542 Total: 2706
Fold: 4 nº treino 2435 nº pos treino 1116 nº neg treino 1319 proporção treino pos 0.458 neg 0.542 Total: 2706
Fold: 4 nº teste 271 nº pos teste 124 nº neg teste 147 proporção teste pos 0.458 neg 0.542 Total: 2706
Fold: 5 nº treino 2435 nº pos treino 1116 nº neg treino 1319 proporção treino pos 0.458 neg 0.542 Total: 2706

10